# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("../Output_data/cities.csv")
cities.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kuandian,40.7286,124.7847,86.774,61,80,2.56,CN,1627101794
1,1,Etah,27.6333,78.6667,96.224,48,64,3.68,IN,1627101794
2,2,Belmonte,-15.8631,-38.8828,66.704,77,100,4.69,BR,1627101794
3,3,Kānker,20.2719,81.4931,74.246,92,100,4.27,IN,1627101795
4,4,Saint-Paul-lès-Dax,43.7272,-1.0516,62.672,97,10,1.03,FR,1627101795


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [47]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
locations = cities[["Lat", "Lng"]].astype(float)
humidity = cities["Humidity"].astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
### Weather Conditions ###
cities_narrow = cities.loc[(cities['Max Temp']<80)&(cities['Max Temp']>75)&(cities['Wind Speed']<10)&(cities['Cloudiness']==0)]
cities_narrow

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
148,148,Genç,38.7507,40.5603,75.560,43,0,6.17,TR,1627101835
170,170,Novoil'insk,51.6829,108.7143,77.288,37,0,3.32,RU,1627101841
200,200,Avarua,-21.2078,-159.7750,75.254,73,0,3.60,CK,1627101447
258,258,Krasnyye Barrikady,46.2045,47.8535,77.000,61,0,4.00,RU,1627101870
310,310,Abu Samrah,35.3029,37.1841,77.540,63,0,4.03,SY,1627101887
446,446,Kangāvar,34.5043,47.9653,78.278,23,0,0.76,IR,1627101934
474,474,Assiut,27.1810,31.1837,78.980,36,0,4.37,EG,1627101943
475,475,Marsá Maţrūḩ,31.3525,27.2453,77.198,69,0,3.60,EG,1627101943
481,481,Awjilah,29.1081,21.2869,76.892,47,0,4.07,LY,1627101945


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = pd.DataFrame({})
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
for index,row in cities_narrow.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel = requests.get(base_url, params=params).json()    
    try:
        hotel_df.loc[index,'Hotel Name'] = hotel["results"][0]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index,'Hotel Name'] = "No Hotel within 5000 meters"
hotel_df 

,Hotel Name
148,Genc Ogretmenevi
170,No Hotel within 5000 meters
200,Paradise Inn
258,Baza Otdykha
310,No Hotel within 5000 meters
446,محمد جوادرضایی
474,Florence Hotel Assiut
475,Beau Site
481,No Hotel within 5000 meters


In [9]:
hotels = pd.concat([cities_narrow,hotel_df],axis=1).reset_index()
hotels

,index,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,148,148,Genç,38.7507,40.5603,75.560,43,0,6.17,TR,1627101835,Genc Ogretmenevi
1,170,170,Novoil'insk,51.6829,108.7143,77.288,37,0,3.32,RU,1627101841,No Hotel within 5000 meters
2,200,200,Avarua,-21.2078,-159.7750,75.254,73,0,3.60,CK,1627101447,Paradise Inn
3,258,258,Krasnyye Barrikady,46.2045,47.8535,77.000,61,0,4.00,RU,1627101870,Baza Otdykha
4,310,310,Abu Samrah,35.3029,37.1841,77.540,63,0,4.03,SY,1627101887,No Hotel within 5000 meters
5,446,446,Kangāvar,34.5043,47.9653,78.278,23,0,0.76,IR,1627101934,محمد جوادرضایی
6,474,474,Assiut,27.1810,31.1837,78.980,36,0,4.37,EG,1627101943,Florence Hotel Assiut
7,475,475,Marsá Maţrūḩ,31.3525,27.2453,77.198,69,0,3.60,EG,1627101943,Beau Site
8,481,481,Awjilah,29.1081,21.2869,76.892,47,0,4.07,LY,1627101945,No Hotel within 5000 meters


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels.iterrows()]
marker_locations = hotels[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
locations = cities[["Lat", "Lng"]].astype(float)
humidity = cities["Humidity"].astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
marker_layer = gmaps.marker_layer(marker_locations,
                                        info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))